In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision

In [2]:
# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [3]:
# CIFAR-10的类别
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

In [5]:
# 加载数据集
trainset = datasets.CIFAR10(root='../data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='../data', train=False, download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
trainloader = DataLoader(trainset, batch_size=32, shuffle=True, num_workers=16)
testloader = DataLoader(testset, batch_size=32, shuffle=False, num_workers=16)

In [7]:
# 定义CNN模型
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [9]:
model = SimpleCNN()
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)


In [11]:
num_epochs = 50

# 记录训练过程中的损失和准确率
train_loss_history = []
train_acc_history = []

# 训练循环
for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0
    
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
        if i % 2000 == 1999:
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
            running_loss = 0.0
    
    epoch_loss = running_loss / len(trainloader)
    epoch_acc = 100 * correct / total
    train_loss_history.append(epoch_loss)
    train_acc_history.append(epoch_acc)
    print(f'Epoch {epoch + 1} finished, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%')

print('Finished Training')

Epoch 1 finished, Loss: 2.2180, Accuracy: 18.24%
Epoch 2 finished, Loss: 1.8030, Accuracy: 33.69%
Epoch 3 finished, Loss: 1.5536, Accuracy: 43.43%
Epoch 4 finished, Loss: 1.4334, Accuracy: 47.98%
Epoch 5 finished, Loss: 1.3476, Accuracy: 51.48%
Epoch 6 finished, Loss: 1.2872, Accuracy: 54.07%
Epoch 7 finished, Loss: 1.2294, Accuracy: 56.14%
Epoch 8 finished, Loss: 1.1843, Accuracy: 58.07%
Epoch 9 finished, Loss: 1.1450, Accuracy: 59.53%
Epoch 10 finished, Loss: 1.1038, Accuracy: 60.91%
Epoch 11 finished, Loss: 1.0714, Accuracy: 62.20%
Epoch 12 finished, Loss: 1.0355, Accuracy: 63.36%
Epoch 13 finished, Loss: 1.0067, Accuracy: 64.53%
Epoch 14 finished, Loss: 0.9764, Accuracy: 65.71%
Epoch 15 finished, Loss: 0.9502, Accuracy: 66.66%
Epoch 16 finished, Loss: 0.9257, Accuracy: 67.47%
Epoch 17 finished, Loss: 0.8984, Accuracy: 68.26%
Epoch 18 finished, Loss: 0.8750, Accuracy: 69.40%
Epoch 19 finished, Loss: 0.8535, Accuracy: 70.11%
Epoch 20 finished, Loss: 0.8299, Accuracy: 71.15%
Epoch 21 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total:.2f}%')

Accuracy of the network on the 10000 test images: 63.08%


In [13]:
torch.save(model.state_dict(), 'simple_cnn.pth')

In [14]:
import torch
import torch.nn as nn
from torch.linalg import svdvals

def compute_lipschitz_constant(model):
    """
    Compute the Lipschitz constant of a PyTorch model.
    Assumes:
        - All layers are either Conv2d or Linear (no normalization layers).
        - No activation functions (or they are Lipschitz-1 like ReLU).
        - Pooling layers (MaxPool, AvgPool) have Lipschitz constant = 1.
    Returns:
        Lipschitz constant (float)
    """
    lipschitz = 1.0
    
    for layer in model.modules():
        if isinstance(layer, (nn.Conv2d, nn.Linear)):
            # Get the weight matrix (for Conv2d, reshape to 2D matrix)
            W = layer.weight
            
            if isinstance(layer, nn.Conv2d):
                # For Conv2d, reshape to (out_channels, in_channels * kernel_h * kernel_w)
                in_channels = layer.in_channels
                out_channels = layer.out_channels
                kernel_size = layer.kernel_size
                
                # Reshape W to (out_channels, in_channels * kernel_h * kernel_w)
                W_reshaped = W.view(out_channels, -1)
            else:
                # For Linear, already in (out_features, in_features)
                W_reshaped = W
            
            # Compute spectral norm (largest singular value)
            singular_values = svdvals(W_reshaped)
            spectral_norm = singular_values.max().item()
            
            lipschitz *= spectral_norm
    
    return lipschitz

In [15]:
# 计算Lipschitz常数
lipschitz = compute_lipschitz_constant(model)
print(f"Lipschitz constant: {lipschitz}")

Lipschitz constant: 259.0588772652261
